<a href="https://colab.research.google.com/github/iam-pattan/DL-lab/blob/main/Lab_9(Speech).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/PacktPublishing/Python-Deep-Learning-Cookbook/blob/master/Chapter09/Chapter%209%20-%20Identifying%20speakers%20with%20voice%20recognition.ipynb

https://github.com/aravindpai/Speech-Recognition/blob/master/Speech%20Recognition.ipynb

# CLASS CODE

In [1]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf

In [ ]:
# !wget https://zenodo.org/record/1342401/files/Jakobovski/free-spoken-digit-dataset-v1.0.8.zip
# !unzip free-spoken-digit-dataset-v1.0.8.zip

--2021-04-30 10:05:17--  https://zenodo.org/record/1342401/files/Jakobovski/free-spoken-digit-dataset-v1.0.8.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7273063 (6.9M) [application/octet-stream]
Saving to: ‘free-spoken-digit-dataset-v1.0.8.zip.3’

free-spoken-digit-d 100%[===================>]   6.94M  2.83MB/s    in 2.5s    

2021-04-30 10:05:22 (2.83 MB/s) - ‘free-spoken-digit-dataset-v1.0.8.zip.3’ saved [7273063/7273063]

Archive:  free-spoken-digit-dataset-v1.0.8.zip
e9e1155aba3a0c632823b813aa74918807644509
replace Jakobovski-free-spoken-digit-dataset-e9e1155/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
SEED = 123
datapath = "/content/Jakobovski-free-spoken-digit-dataset-e9e1155/recordings/"
'/content/Jakobovski-free-spoken-digit-dataset-e9e1155'

'/content/Jakobovski-free-spoken-digit-dataset-e9e1155'

In [ ]:
files = glob.glob(datapath+"*.wav")
len(files)

1500

In [ ]:
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 1200
# Validation examples: 300


In [ ]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    if label not in labels:
        labels.append(label)
print(labels)

['nicolas', 'theo', 'jackson']


In [ ]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [ ]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [ ]:
def generator(number):
    i=0
    while i< number:
        i = i+1
        if i%2==0:
            yield i

In [ ]:
datagen = generator(10)

In [ ]:
next(datagen)

2

In [ ]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [ ]:
train_datagen = batch_generator(X_train, batch_size=32)
val_datagen = batch_generator(X_val, batch_size = 4)

In [ ]:
#Model DEFN
input_shape = (n_features, max_length)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('SPEAKER_best_model_.hdf5', save_best_only=True),
            tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]

In [ ]:
# help(model.fit)

In [ ]:
history = model.fit(
    train_datagen, 
    steps_per_epoch=10,
    epochs=3,
    validation_data=val_datagen,
    validation_steps=5,
    validation_freq=1,
    verbose=1
    # callbacks = callbacks
)

In [ ]:
def testSample(filename,model):
  # Read the WAV file and transform it into mfcc
  print(filename)
  max_length = 80
  wave, sr = librosa.load(filename, mono=True)
  mfcc = librosa.feature.mfcc(wave, sr)
  if(len(mfcc[0]) < 60 ):
    mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
    X = np.array([np.array(mfcc)])
    label = filename.split('/')[-1].split('_')[1]
    pred = model.predict(X)
    pred = np.round(pred,decimals=1)
    pred = label_binarizer.inverse_transform(pred)
    print("Actual Label: ",label)
    print("Predicted Label: ",pred[0])
  else:
    print("Incompatible Audio")

In [ ]:
testSample(files[300],model)

/content/Jakobovski-free-spoken-digit-dataset-e9e1155/recordings/0_nicolas_21.wav
Actual Label:  nicolas
Predicted Label:  nicolas


In [ ]:
# APPENDIX 
# code for loading the entire data into memory as numpy
random.shuffle(files)
X, y = [], []
checkpoint = 0
for i in range(len(files)):
    wav = files[i]
    wave, sr = librosa.load(wav, mono=True)
    mfcc = librosa.feature.mfcc(wave, sr)
    if(len(mfcc[0]) < 80 ):
      mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
      X.append(np.array(mfcc))
      label = wav.split('/')[-1].split('_')[1]
      y.append(label)
      checkpoint+=1
    if(checkpoint%100 ==0):
      print("Current Data count Processed: ",checkpoint)

Current Data count Processed:  100
Current Data count Processed:  200
Current Data count Processed:  300
Current Data count Processed:  400
Current Data count Processed:  500
Current Data count Processed:  600
Current Data count Processed:  700
Current Data count Processed:  800
Current Data count Processed:  900
Current Data count Processed:  1000
Current Data count Processed:  1100
Current Data count Processed:  1200
Current Data count Processed:  1300
Current Data count Processed:  1400


In [ ]:
X = np.array(X)
y = one_hot_encode(y)

# Splitting the dataset into train and validation set 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# CODE FOR DIGIT RECOGNITION USING CNN

In [2]:
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git

Cloning into 'free-spoken-digit-dataset'...
remote: Enumerating objects: 4182, done.
remote: Counting objects: 100% (510/510), done.
remote: Compressing objects: 100% (510/510), done.
remote: Total 4182 (delta 2), reused 502 (delta 0), pack-reused 3672
Receiving objects: 100% (4182/4182), 30.44 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [3]:
pth = '/content/free-spoken-digit-dataset/recordings/*'

In [4]:
files = glob.glob(pth + ".wav")
len(files)

3000

In [5]:
X_train, X_val = train_test_split(files, test_size=0.2, random_state=7)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 2400
# Validation examples: 600


In [6]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    if label not in labels:
        labels.append(label)
print(labels)

['yweweler', 'george', 'lucas', 'theo', 'jackson', 'nicolas']


In [7]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [8]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [9]:
n_classes

6

In [10]:
def generator(number):
    i=0
    while i< number:
        i = i+1
        if i%2==0:
            yield i

In [11]:
datagen = generator(10)

In [12]:
next(datagen)

2

In [13]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [14]:
train_datagen = batch_generator(X_train, batch_size=32)
val_datagen = batch_generator(X_val, batch_size = 4)

In [21]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, TimeDistributed
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

#Model DEFN
input_shape = (n_features, max_length,1)
model = Sequential([
    layers.Input(shape=input_shape),
    preprocessing.Resizing(32, 32), 
    # norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(n_classes, activation='softmax')
    ])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_5 (Resizing)        (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)              

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
histroy = model.fit(
            train_datagen, 
            steps_per_epoch=10,
            epochs=3,
            validation_data=val_datagen,
            validation_steps=5,
            validation_freq=1,
            verbose=1)

Epoch 1/3
10/10 [==============================] - 42s 1s/step - loss: 7.0333 - accuracy: 0.2921 - val_loss: 0.4956 - val_accuracy: 0.8000
Epoch 2/3
10/10 [==============================] - 12s 1s/step - loss: 1.2473 - accuracy: 0.5289 - val_loss: 0.8708 - val_accuracy: 0.7000
Epoch 3/3
10/10 [==============================] - 12s 1s/step - loss: 0.9232 - accuracy: 0.6724 - val_loss: 0.3255 - val_accuracy: 0.9500


In [28]:
testSample(files[300],model)

/content/free-spoken-digit-dataset/recordings/8_nicolas_0.wav
Actual Label:  nicolas
Predicted Label:  nicolas


# Explorations here(Don't mind)

In [ ]:
import csv
import io
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, TimeDistributed
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow_io as tfio
import matplotlib.pyplot as plt
maxData = 30
block_length = 0.050#->500ms
voice_max_length = int(10/block_length)#->2s
print("voice_max_length:", voice_max_length)
def audioToTensor(filepath):
    audio_binary = tf.io.read_file(filepath)
    audio, audioSR = tf.audio.decode_wav(audio_binary)
    audioSR = tf.get_static_value(audioSR)
    audio = tf.squeeze(audio, axis=-1)
    audio_length = int(audioSR * block_length)#20-> 50ms 40 -> 25ms
    frame_step = int(audioSR * 0.008)# 128 when rate is 1600 -> 8ms
    audio_clean = tf.constant([], tf.float32)
    audio_length_clean = audioSR//20#50ms
    for i in range(0, len(audio), audio_length_clean):
        audio_slice = audio[i:i+audio_length_clean]
        position = tfio.experimental.audio.trim(audio_slice, axis=0, epsilon=0.065)
        start, stop=position[0], position[1]
        if stop-start<5:
            continue
        audio_slice = audio_slice[start:stop]
        audio_clean = tf.concat([audio_clean, audio_slice], 0)
if len(audio_clean)<audio_length*voice_max_length:
        audio = tf.concat([np.zeros([audio_length*voice_max_length-len(audio)]), audio], 0)
    else:
        audio = audio[-(audio_length*voice_max_length):]
spectrogram = tf.signal.stft(audio, frame_length=1024, frame_step=frame_step)
    spectrogram = (tf.math.log(tf.abs(tf.math.real(spectrogram)))/tf.math.log(tf.constant(10, dtype=tf.float32))*20)-60
    spectrogram = tf.where(tf.math.is_nan(spectrogram), tf.zeros_like(spectrogram), spectrogram)
    spectrogram = tf.where(tf.math.is_inf(spectrogram), tf.zeros_like(spectrogram), spectrogram)
    voice_length, voice = 0, []
    nb_part = len(audio)//audio_length
    part_length = len(spectrogram)//nb_part
    partsCount = len(range(0, len(spectrogram)-part_length, int(part_length/2)))
    parts = np.zeros((partsCount, part_length, 513))
    for i, p in enumerate(range(0, len(spectrogram)-part_length, int(part_length/2))):
        part = spectrogram[p:p+part_length]
        parts[i] = part
    return parts
testParts = audioToTensor('clips/common_voice_en_699711.wav')
print(testParts.shape)
def loadDataFromFile(filepath):
    dataVoice, dataString = [], []
    string_max_lenght = 0
    with open(filepath) as tsvfile:
      reader = csv.reader(tsvfile, delimiter='\t')
      next(reader)#skip header
      for row in reader:
        if len(dataString)>maxData:
            break
        sentence = row[2].replace(".", "")
        wordList = ("start " + sentence + " end").split(" ")
        sentence = row[2].replace(".", "")
        if(len(wordList)<5):
            continue
        print(row[1], row[2], wordList)
        string_max_lenght = max(len(wordList), string_max_lenght)
        dataString.append(wordList)
        dataVoice.append(row[1].replace(".mp3", '.wav'))
    return dataVoice, dataString, string_max_lenght
dataVoice, dataString, string_max_lenght = loadDataFromFile('train.tsv')
print("voice_max_length: ", voice_max_length)
print("string_max_lenght: ", string_max_lenght)
tokenizer = Tokenizer(num_words=2000, lower=True, oov_token="<rare>")
tokenizer.fit_on_texts(dataString)
with io.open('tokenizer.txt', 'w', encoding='utf-8') as f:
    for word, index in tokenizer.word_index.items():
        f.write(word + ":" + str(index) + "\n")
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
def prepareData(dataString, dataVoice):
    X_voice, X_string, Y_string = list(), list(), list()
    for i, seq in enumerate(dataString):
        voice =  dataVoice[i]
        seq = tokenizer.texts_to_sequences([seq])[0]
        for j in range(1, len(seq)):
            in_seq, out_seq = seq[:j], seq[:j+1]
            in_seq = pad_sequences([in_seq], maxlen=string_max_lenght-1)[0]
            out_seq = pad_sequences([out_seq], maxlen=string_max_lenght-1)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X_voice.append(voice)
            X_string.append(in_seq)
            Y_string.append(out_seq)
    return X_voice, X_string, Y_string
X_voice, X_string, Y_string = prepareData(dataString, dataVoice)
print("len(X_voice): ", len(X_voice))
class MySequence(tf.keras.utils.Sequence):
    def __init__(self, x_voice, x_string, y_string, batch_size):
        self.x_voice, self.x_string, self.y_string = x_voice, x_string, y_string
        self.batch_size = batch_size
    def __len__(self):
        return int(len(self.x_voice) / self.batch_size)
    def __getitem__(self, idx):
        batch_x_string = self.x_string[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y_string = self.y_string[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x_voice = []
        for i in range(0, batch_size):
            voice = audioToTensor('clips/' + self.x_voice[idx * self.batch_size + i])
            batch_x_voice.append(voice)
        return [np.array(batch_x_voice), np.array(batch_x_string)], np.array(batch_y_string)
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
latent_dim=64
encoder_inputs = Input(shape=(testParts.shape[0], None, None, 1))
preprocessing = TimeDistributed(preprocessing.Resizing(6, 129))(encoder_inputs)
normalization = TimeDistributed(BatchNormalization())(preprocessing)
conv2d = TimeDistributed(Conv2D(34, 3, activation='relu'))(normalization)
conv2d = TimeDistributed(Conv2D(64, 3, activation='relu'))(conv2d)
maxpool = TimeDistributed(MaxPooling2D())(conv2d)
dropout = TimeDistributed(Dropout(0.25))(maxpool)
flatten = TimeDistributed(Flatten())(dropout)
encoder_lstm = LSTM(units=latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(flatten)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(string_max_lenght-1))
dec_emb_layer = Embedding(vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary(line_length=200)
tf.keras.utils.plot_model(model, to_file='model_sentence.png', show_shapes=True)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
batch_size = 32
epochs = 30
model.fit(MySequence(X_voice, X_string, Y_string, batch_size), epochs=epochs, steps_per_epoch=len(X_string)//batch_size)
model.save_weights('model_sentence.h5')
model.save("model_sentence")